In [1]:
import pickle
from tqdm import tqdm
import sys
import hashlib

# TO CHANGE
BASEDIR = "../"
# TO CHNAGE
sys.path.insert(0, BASEDIR)

from src.utils.data_structs import TripletCreator, NodeCreator, Relation, NODES_TYPES_MAP, RELATIONS_TYPES_MAP
from src.db_drivers.graph_driver.connectors.Neo4jConnector import Neo4jConnector
from src.db_drivers.graph_driver import GraphDBConnectionConfig
NEO4J_URL ="bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PWD = "password"
GRAPH_DB_NAME = 'DiaasqGigachat'

/home/dzigen/Desktop/PersonalAI/pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = GraphDBConnectionConfig(uri=NEO4J_URL, params={'user': NEO4J_USER, 'pwd': NEO4J_PWD, 'db_name': GRAPH_DB_NAME})

In [3]:
graph_db = Neo4jConnector(config)

In [4]:
def create_id(seed: str) -> str:
    return hashlib.md5(seed.encode()).hexdigest()

In [5]:
raw_triplets = list(graph_db.execute_query("MATCH (n1)-[rel]->(n2) RETURN n1, rel, n2"))
formated_triplets = graph_db.parse_query_output(raw_triplets )
print("formated_triplets amount: ", len(formated_triplets))

formated_triplets amount:  211542


In [6]:
unique_nodes = dict()
unique_relations = dict()
for triplet in tqdm(formated_triplets):
    sn_str_id = create_id(NodeCreator.stringify(triplet.start_node)[1])
    en_str_id = create_id(NodeCreator.stringify(triplet.end_node)[1])
    unique_nodes[sn_str_id] = triplet.start_node
    unique_nodes[en_str_id] = triplet.end_node
    unique_relations[triplet.relation.id] = triplet

100%|██████████| 211542/211542 [00:01<00:00, 195218.98it/s]


In [7]:
print(len(unique_nodes), len(unique_relations))

54051 211542


In [8]:
for str_id, node in tqdm(unique_nodes.items()):
    graph_db.execute_query(f'MATCH (n) WHERE elementId(n) = "{node.id}" SET n.str_id = "{str_id}";')

100%|██████████| 54051/54051 [47:26<00:00, 18.99it/s]  


In [8]:
for rel_id, triplet in tqdm(unique_relations.items()):
    graph_db.execute_query(f'MATCH (n1)-[rel]->(n2) WHERE elementId(n1) = "{triplet.start_node.id}" AND elementId(n2) = "{triplet.end_node.id}" AND elementId(rel) = "{triplet.relation.id}" SET rel.str_id = "{triplet.id}";')

100%|██████████| 211542/211542 [3:13:01<00:00, 18.26it/s]  


In [9]:
raw_triplets = list(graph_db.execute_query("MATCH (n1)-[rel]->(n2) RETURN n1, rel, n2"))
formated_triplets = graph_db.parse_query_output(raw_triplets)
print("formated_triplets amount: ", len(formated_triplets))

formated_triplets amount:  211542


In [10]:
with open(f'../../{GRAPH_DB_NAME}.pickle', 'wb') as f:
    pickle.dump(formated_triplets, f)